<a href="https://colab.research.google.com/github/KevinYih/BigDataDemo/blob/main/Copy_of_SVD_Netflix_movie_ratings_by_Kevin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SVD - Netflix movie ratings

Adapted from: https://colab.research.google.com/github/ashishdasari148/Recommender-Systems-using-Collaborative-Filtering/blob/master/Netflix.ipynb#scrollTo=qX7Nk4tUGZND

In [1]:
!echo '{"username":"kevinyih","key":"9a18a73669b892b2ac58ebb67dc96545"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d netflix-inc/netflix-prize-data

/bin/bash: line 1: /root/.kaggle/kaggle.json: No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data
License(s): other
 98% 669M/683M [00:05<00:00, 192MB/s]
100% 683M/683M [00:05<00:00, 138MB/s]


In [2]:
import os
import zipfile
with zipfile.ZipFile('netflix-prize-data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
import pandas as pd

import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
dir_name = 'data'

In [4]:
# Converting the txt files into csv files
files = ['combined_data_1.txt','combined_data_2.txt','combined_data_3.txt','combined_data_4.txt']
for j in files:
    file1 = open(os.path.join(dir_name, j),"r+")
    print(j + " opened...")
    file2 = open(os.path.join(dir_name, j + '.csv'),"a")
    for i in file1.readlines():
        temp = re.match("(\d*)[:]", i)
        if(temp):
            movienumber = temp.group(1)
        else:
            file2.write(str(movienumber)+","+i)
    file1.close()
    file2.close()
    print(j + " closed...")

combined_data_1.txt opened...
combined_data_1.txt closed...
combined_data_2.txt opened...
combined_data_2.txt closed...
combined_data_3.txt opened...
combined_data_3.txt closed...
combined_data_4.txt opened...
combined_data_4.txt closed...


In [5]:
movie_titles = pd.read_csv(os.path.join(dir_name, 'movie_titles.csv'), encoding = 'ISO-8859-1', header = None, names=['Movie', 'Year of Release', 'Title'], usecols=[0, 1, 2])
movie_titles

,Movie,Year of Release,Title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [6]:
## appending all the csv files of rating data into a single dataframe
dfs = []
for i in ['combined_data_1.txt.csv','combined_data_2.txt.csv','combined_data_3.txt.csv','combined_data_4.txt.csv']:
    df_ = pd.read_csv(os.path.join(dir_name, i), header=None, names=['Movie', 'User', 'Rating', 'Date'], usecols=[0, 1, 2, 3])
    df_ = df_[['Movie','User','Rating']]
    dfs.append(df_)
df_all = pd.concat(dfs, ignore_index=True)

In [7]:
print("number of ratings:", len(df_all))

number of ratings: 100480507


In [8]:
# search for your favorite movie

indices = movie_titles.Title.str.contains('dinosaur', flags=re.IGNORECASE)
movie_titles[indices]
# 14691    The Matrix
# 3925     The Matrix: Reloaded
# 16767    The Matrix: Revisited
# 468      The Matrix: Revolutions
# 397      A Night in Casablanca
# 10661    Tokyo Story
# 963      A Streetcar Named Desire
# 6099     Apocalypse Now
# 4253    Kind Hearts and Coronets
# 2412     Harry Potter and the Prisoner of Azkaban: Bonu...
# 11443    Harry Potter and the Chamber of Secrets



,Movie,Year of Release,Title
0,1,2003.0,Dinosaur Planet
650,651,1997.0,The Rainbow Fish / Dazzle the Dinosaur
693,694,2000.0,When Dinosaurs Roamed America
1172,1173,1999.0,Walking with Dinosaurs
1882,1883,1960.0,Dinosaurus!
4099,4100,2000.0,Dinosaur
5301,5302,2003.0,Chased by Dinosaurs: Three Walking with Dinosa...
8841,8842,1999.0,Walking with Dinosaurs: Bonus Material
13585,13586,2001.0,Allosaurus: A Walking with Dinosaurs Special
13890,13891,1991.0,Nymphoid Barbarian in Dinosaur Hell


In [9]:
# choosing random movies
random_movie_ids = set(df_all.sample(frac=.0000001).Movie.unique())
print(random_movie_ids)

# or choosing your favorite
favorites_movie_ids = set({1, 651, 694, 1173, 1883, 4100, 5302, 8842, 13586, 13891, 15648 })
print(favorites_movie_ids)


{13602, 17253, 16390, 3148, 17612, 8782, 2674, 14103, 13081, 11804}
{15648, 1, 13891, 4100, 8842, 651, 13586, 1173, 694, 5302, 1883}


In [10]:
df_all[df_all.Movie == 17339 ]

,Movie,User,Rating
98172243,17339,1734805,3
98172244,17339,364518,4
98172245,17339,1392773,3
98172246,17339,716091,4
98172247,17339,2059652,4
...,...,...,...
98243197,17339,702455,3
98243198,17339,1536433,3
98243199,17339,1704768,3
98243200,17339,2486011,4


In [11]:
# saving the merged file to drive for further use.
movie_ids = random_movie_ids.union(favorites_movie_ids)

df_all.to_csv(path_or_buf=os.path.join(dir_name, 'data.txt.csv'),index=False)

In [ ]:
# loading the saved csv into dataframe
df_all = pd.read_csv(os.path.join(dir_name, 'data.txt.csv'))

In [12]:
print("number of ratings:", len(df_all))

number of ratings: 100480507


In [13]:
df_movie_ids = df_all[df_all.Movie.isin(movie_ids)]
print("Unique movie ids in the dataframe:", (df_movie_ids.Movie.unique()))
print("Number of unique user ids:", len(df_movie_ids.User.unique()))

Unique movie ids in the dataframe: [    1   651   694  1173  1883  2674  3148  4100  5302  8782  8842 11804
 13081 13586 13602 13891 14103 15648 16390 17253 17612]
Number of unique user ids: 294930


In [14]:
df_ = df_movie_ids.sample(frac=0.001)
len(df_)

556

In [15]:
# create a dataframe that only contains rating for a subset of users and
# a subset of movies
user_ids = set(df_.User.unique())
df_small = df_all[(df_all.User.isin(user_ids) & df_all.Movie.isin(movie_ids))]
print("Unique movie ids in the dataframe:", (df_small.Movie.unique()))
print("Number of unique user ids:", len(df_small.User.unique()))
print("Number of ratings:", len(df_small))

Unique movie ids in the dataframe: [    1   651   694  1173  1883  2674  3148  4100  5302  8782  8842 11804
 13081 13586 13602 13891 14103 15648 16390 17612]
Number of unique user ids: 555
Number of ratings: 1424


In [17]:
itom = list(sorted(movie_ids))
mtoi = {m: i for i, m in enumerate(itom)}
itou = list(sorted(user_ids))
utoi = {u: i for i, u in enumerate(itou)}

In [18]:
user_movie_rating = np.zeros((len(itou), len(itom)))

for index, row in df_small.iterrows():
  movie_id = row.Movie
  movie_index = mtoi[movie_id]

  user_id = row.User
  user_index = utoi[user_id]

  user_movie_rating[user_index, movie_index] = row.Rating

In [19]:
user_movie_rating.shape

(555, 21)

In [ ]:
# more efficient way of building a user-movie rating matrix
um = df_small.pivot_table(index='User', columns='Movie', values='Rating').values
um.shape

In [ ]:
user_movie_rating[:10,]

In [20]:
# compute the SVD of the matrix
A = user_movie_rating
U, S, Vt = np.linalg.svd(A, full_matrices=False)

In [21]:
print(U.shape, S.shape, Vt.shape)

(555, 21) (21,) (21, 21)


In [ ]:
A_hat = U @ np.diag(S) @ Vt

In [ ]:
np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat[:10,])

In [ ]:
sum([a*a for a in S[:3]])/sum([a*a for a in S])

0.8748565522564679

In [ ]:
r = 8
A_hat_r = U[:,:r] @ np.diag(S[:r]) @ Vt[:r,:]
np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat_r[:10,])